In [3]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

class Solution:
    """The solution of Port Scheduling Problem.
    Contains vessel assignments, tugboat assignments, and timing information.
    
    Attributes:
        vessel_assignments (dict): Maps vessel_id to (berth_id, start_time) tuple. 
                                 None if vessel is unassigned.
        tugboat_inbound_assignments (dict): Maps vessel_id to list of (tugboat_id, start_time) tuples for inbound service.
        tugboat_outbound_assignments (dict): Maps vessel_id to list of (tugboat_id, start_time) tuples for outbound service.
    """
    def __init__(self, vessel_assignments: dict = None, 
                 tugboat_inbound_assignments: dict = None, 
                 tugboat_outbound_assignments: dict = None):
        self.vessel_assignments = vessel_assignments or {}
        self.tugboat_inbound_assignments = tugboat_inbound_assignments or {}
        self.tugboat_outbound_assignments = tugboat_outbound_assignments or {}

    def __str__(self) -> str:
        result = "Port Scheduling Solution:\n"
        result += "Vessel Assignments:\n"
        for vessel_id, assignment in self.vessel_assignments.items():
            if assignment is not None:
                berth_id, start_time = assignment
                result += f"  Vessel {vessel_id}: Berth {berth_id}, Start Time {start_time}\n"
            else:
                result += f"  Vessel {vessel_id}: Unassigned\n"
        
        result += "Inbound Tugboat Services:\n"
        for vessel_id, services in self.tugboat_inbound_assignments.items():
            if services:
                service_str = ", ".join([f"Tugboat {tug_id} at time {start_time}" 
                                       for tug_id, start_time in services])
                result += f"  Vessel {vessel_id}: {service_str}\n"
        
        result += "Outbound Tugboat Services:\n"
        for vessel_id, services in self.tugboat_outbound_assignments.items():
            if services:
                service_str = ", ".join([f"Tugboat {tug_id} at time {start_time}" 
                                       for tug_id, start_time in services])
                result += f"  Vessel {vessel_id}: {service_str}\n"
        
        return result

def load_case_data(filename):
    """Load case data from txt file."""
    data = {}
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#'):
                if '=' in line:
                    key, value = line.split('=', 1)
                    key = key.strip()
                    value = value.strip()
                    
                    # Parse different data types
                    if value.startswith('[') and value.endswith(']'):
                        # List data
                        value = value[1:-1]  # Remove brackets
                        if ',' in value:
                            data[key] = [float(x.strip()) if '.' in x else int(x.strip()) 
                                       for x in value.split(',')]
                        else:
                            data[key] = [float(value) if '.' in value else int(value)]
                    else:
                        # Single value
                        try:
                            data[key] = float(value) if '.' in value else int(value)
                        except ValueError:
                            data[key] = value
    return data

def solve_port_scheduling_gurobi(data_file):
    """Solve simplified port scheduling problem using Gurobi."""
    
    # Load data
    data = load_case_data(data_file)
    
    # Extract parameters
    I = data['vessel_num']  # Number of vessels
    J = data['berth_num']   # Number of berths  
    K = data['tugboat_num'] # Number of tugboats
    T = data['time_periods'] # Time periods
    
    # Vessel parameters
    S = data['vessel_sizes']  # S_i: vessel levels
    ETA = data['vessel_etas']  # ETA_i
    D = data['vessel_durations']  # D_i
    tau_in = data['tau_in']  # τ^in_i
    tau_out = data['tau_out']  # τ^out_i
    Delta_early = data['Delta_early']  # Δ^early_i
    Delta_late = data['Delta_late']  # Δ^late_i
    alpha = data['alpha']  # α_i
    beta = data['beta']  # β_i
    gamma = data['gamma']  # γ_i
    
    # Berth parameters
    C = data['berth_capacities']  # C_j: berth levels
    
    # Tugboat parameters
    G = data['tugboat_capacities']  # G_k: tugboat levels
    c_k = data['c_k']  # c_k: tugboat costs
    
    # System parameters
    rho_in = data['rho_in']  # ρ^in
    rho_out = data['rho_out']  # ρ^out
    epsilon_time = data['epsilon_time']  # ε_time
    M = data['M']  # Big number
    lambda_1, lambda_2, lambda_3, lambda_4 = data['lambda_1'], data['lambda_2'], data['lambda_3'], data['lambda_4']
    
    # Create model
    model = gp.Model("SimplifiedPortScheduling")
    
    # Decision variables
    # x[i,j,t]: vessel i starts berthing at berth j at time t
    x = model.addVars(I, J, T, vtype=GRB.BINARY, name="x")
    
    # y_in[i,k,t]: tugboat k starts inbound service for vessel i at time t
    y_in = model.addVars(I, K, T, vtype=GRB.BINARY, name="y_in")
    
    # y_out[i,k,t]: tugboat k starts outbound service for vessel i at time t  
    y_out = model.addVars(I, K, T, vtype=GRB.BINARY, name="y_out")
    
    # u_early[i], u_late[i]: ETA deviation variables
    u_early = model.addVars(I, vtype=GRB.CONTINUOUS, name="u_early")
    u_late = model.addVars(I, vtype=GRB.CONTINUOUS, name="u_late")
    
    # Objective function components
    # Z1: Unserved vessel penalty
    Z1 = gp.quicksum(M * alpha[i] * (1 - gp.quicksum(x[i,j,t] for j in range(J) for t in range(T))) 
                     for i in range(I))
    
    # Z2: Total port time cost
    Z2 = gp.quicksum(alpha[i] * beta[i] * (
        gp.quicksum((t + tau_out[i]) * y_out[i,k,t] for k in range(K) for t in range(T)) - 
        gp.quicksum(t * y_in[i,k,t] for k in range(K) for t in range(T))
    ) for i in range(I))
    
    # Z3: ETA deviation cost
    Z3 = gp.quicksum(alpha[i] * gamma[i] * (u_early[i] + u_late[i]) for i in range(I))
    
    # Z4: Tugboat utilization cost
    Z4 = gp.quicksum(c_k[k] * (tau_in[i] * y_in[i,k,t] + tau_out[i] * y_out[i,k,t]) 
                     for k in range(K) for i in range(I) for t in range(T))
    
    # Set objective
    model.setObjective(lambda_1 * Z1 + lambda_2 * Z2 + lambda_3 * Z3 + lambda_4 * Z4, GRB.MINIMIZE)
    
    # Constraints
    
    # Constraint (1): Each vessel can be assigned at most once
    for i in range(I):
        model.addConstr(gp.quicksum(x[i,j,t] for j in range(J) for t in range(T)) <= 1, 
                       name=f"vessel_assignment_{i}")
    
    # Constraint (2): Inbound tugboat service coupling
    for i in range(I):
        model.addConstr(gp.quicksum(y_in[i,k,t] for k in range(K) for t in range(T)) == 
                       gp.quicksum(x[i,j,t] for j in range(J) for t in range(T)), 
                       name=f"inbound_coupling_{i}")
    
    # Constraint (3): Outbound tugboat service coupling  
    for i in range(I):
        model.addConstr(gp.quicksum(y_out[i,k,t] for k in range(K) for t in range(T)) == 
                       gp.quicksum(x[i,j,t] for j in range(J) for t in range(T)), 
                       name=f"outbound_coupling_{i}")
    
    # Constraint (4): Vessel-berth compatibility
    for i in range(I):
        for j in range(J):
            for t in range(T):
                if C[j] < S[i]:
                    model.addConstr(x[i,j,t] == 0, name=f"berth_compatibility_{i}_{j}_{t}")
    
    # Constraint (5): Vessel-tugboat compatibility for inbound
    for i in range(I):
        for k in range(K):
            for t in range(T):
                if G[k] < S[i]:
                    model.addConstr(y_in[i,k,t] == 0, name=f"tugboat_inbound_compatibility_{i}_{k}_{t}")
    
    # Constraint (6): Vessel-tugboat compatibility for outbound
    for i in range(I):
        for k in range(K):
            for t in range(T):
                if G[k] < S[i]:
                    model.addConstr(y_out[i,k,t] == 0, name=f"tugboat_outbound_compatibility_{i}_{k}_{t}")
    
    # Constraint (7): Each vessel at most one inbound service
    for i in range(I):
        model.addConstr(gp.quicksum(y_in[i,k,t] for k in range(K) for t in range(T)) <= 1, 
                       name=f"max_one_inbound_{i}")
    
    # Constraint (8): Each vessel at most one outbound service
    for i in range(I):
        model.addConstr(gp.quicksum(y_out[i,k,t] for k in range(K) for t in range(T)) <= 1, 
                       name=f"max_one_outbound_{i}")
    
    # Constraint (9): Inbound-berthing timing sequence
    for i in range(I):
        berth_start = gp.quicksum(t * x[i,j,t] for j in range(J) for t in range(T))
        inbound_end = gp.quicksum((t + tau_in[i]) * y_in[i,k,t] for k in range(K) for t in range(T))
        vessel_assigned = gp.quicksum(x[i,j,t] for j in range(J) for t in range(T))
        
        model.addConstr(berth_start - inbound_end >= 0, 
                       name=f"inbound_berth_lower_{i}")
        model.addConstr(berth_start - inbound_end <= epsilon_time * vessel_assigned, 
                       name=f"inbound_berth_upper_{i}")
    
    # Constraint (10): Berthing-outbound timing sequence
    for i in range(I):
        outbound_start = gp.quicksum(t * y_out[i,k,t] for k in range(K) for t in range(T))
        berth_end = gp.quicksum((t + D[i]) * x[i,j,t] for j in range(J) for t in range(T))
        vessel_assigned = gp.quicksum(x[i,j,t] for j in range(J) for t in range(T))
        
        model.addConstr(outbound_start - berth_end >= 0, 
                       name=f"berth_outbound_lower_{i}")
        model.addConstr(outbound_start - berth_end <= epsilon_time * vessel_assigned, 
                       name=f"berth_outbound_upper_{i}")
    
    # Constraint (11): Berth capacity constraints
    for j in range(J):
        for t in range(T):
            model.addConstr(gp.quicksum(x[i,j,tau] 
                           for i in range(I) 
                           for tau in range(max(0, t-D[i]+1), t+1)) <= 1, 
                           name=f"berth_capacity_{j}_{t}")
    
    # Constraint (12): Tugboat capacity constraints with preparation time
    for k in range(K):
        for t in range(T):
            model.addConstr(
                gp.quicksum(y_in[i,k,tau] 
                           for i in range(I) 
                           for tau in range(max(0, t-tau_in[i]-rho_in+1), t+1)) +
                gp.quicksum(y_out[i,k,tau] 
                           for i in range(I) 
                           for tau in range(max(0, t-tau_out[i]-rho_out+1), t+1)) <= 1,
                name=f"tugboat_capacity_{k}_{t}")
    
    # Constraint (13): Service time window constraints for inbound
    for i in range(I):
        for k in range(K):
            for t in range(T):
                if t < ETA[i] - Delta_early[i] or t > ETA[i] + Delta_late[i]:
                    model.addConstr(y_in[i,k,t] == 0, name=f"time_window_in_{i}_{k}_{t}")
    
    # Constraint (14): ETA deviation linearization
    for i in range(I):
        inbound_start = gp.quicksum(t * y_in[i,k,t] for k in range(K) for t in range(T))
        model.addConstr(inbound_start == ETA[i] + u_late[i] - u_early[i], 
                       name=f"eta_deviation_{i}")
    
    # Solve the model
    model.setParam('TimeLimit', 60*1)  # 2 minutes time limit
    model.setParam('MIPGap', 0.01)     # 1% optimality gap
    model.optimize()
    
    # Extract and return solution in Solution class format
    if model.Status == GRB.OPTIMAL or model.Status == GRB.TIME_LIMIT:
        print(f"Solution Status: {model.Status}")
        print(f"Objective Value: {model.ObjVal:.2f}")
        
        # Extract vessel assignments
        vessel_assignments = {}
        for i in range(I):
            vessel_assignments[i] = None  # Default to unassigned
            for j in range(J):
                for t in range(T):
                    if x[i,j,t].X > 0.5:
                        vessel_assignments[i] = (j, t)
                        break
                if vessel_assignments[i] is not None:
                    break
        
        # Extract tugboat inbound assignments
        tugboat_inbound_assignments = {}
        for i in range(I):
            services = []
            for k in range(K):
                for t in range(T):
                    if y_in[i,k,t].X > 0.5:
                        services.append((k, t))
            tugboat_inbound_assignments[i] = services
        
        # Extract tugboat outbound assignments
        tugboat_outbound_assignments = {}
        for i in range(I):
            services = []
            for k in range(K):
                for t in range(T):
                    if y_out[i,k,t].X > 0.5:
                        services.append((k, t))
            tugboat_outbound_assignments[i] = services
        
        # Create Solution object
        solution = Solution(
            vessel_assignments=vessel_assignments,
            tugboat_inbound_assignments=tugboat_inbound_assignments,
            tugboat_outbound_assignments=tugboat_outbound_assignments
        )
        
        # Print solution using Solution class format
        print("\n" + str(solution))
        
        # Print objective breakdown
        print("=== OBJECTIVE BREAKDOWN ===")
        z1_val = sum(M * alpha[i] * (1 - sum(x[i,j,t].X for j in range(J) for t in range(T))) 
                     for i in range(I))
        z2_val = sum(alpha[i] * beta[i] * (
            sum((t + tau_out[i]) * y_out[i,k,t].X for k in range(K) for t in range(T)) - 
            sum(t * y_in[i,k,t].X for k in range(K) for t in range(T))
        ) for i in range(I))
        z3_val = sum(alpha[i] * gamma[i] * (u_early[i].X + u_late[i].X) for i in range(I))
        z4_val = sum(c_k[k] * (tau_in[i] * y_in[i,k,t].X + tau_out[i] * y_out[i,k,t].X) 
                     for k in range(K) for i in range(I) for t in range(T))
        
        print(f"Z1 (Unserved penalty): {z1_val:.2f}")
        print(f"Z2 (Port time cost): {z2_val:.2f}")
        print(f"Z3 (ETA deviation cost): {z3_val:.2f}")
        print(f"Z4 (Tugboat cost): {z4_val:.2f}")
        print(f"Total Objective: {model.ObjVal:.2f}")
        
        return solution, model
    else:
        print(f"No solution found. Status: {model.Status}")
        return None, None

if __name__ == "__main__":
    # Solve with updated data file
    solution, model = solve_port_scheduling_gurobi(
        r"C:\Users\gongh\HeurAgenix-TS\HeurAgenix-main\output\psp\data\test_data\100_10_25_T48.txt")

    if solution is not None:
        print(f"\nSolution successfully obtained!")
        print(f"Gurobi model status: {model.Status}")
        # You can now use the solution object for further analysis
        # For example, validate the solution or calculate additional metrics
    else:
        print("Failed to obtain solution.")

Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12450H, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  60
MIPGap  0.01

Optimize a model with 195868 rows, 288200 columns and 2925998 nonzeros
Model fingerprint: 0x44968c74
Variable types: 200 continuous, 288000 integer (288000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e-03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 152693.61316
Presolve removed 193188 rows and 161024 columns
Presolve time: 0.33s
Presolved: 2680 rows, 127176 columns, 1198182 nonzeros
Variable types: 198 continuous, 126978 integer (126977 binary)
Performing another presolve...
Presolve removed 156 rows and 47164 columns

KeyboardInterrupt: 

Exception ignored in: 'gurobipy._core.logcallbackstub'
Traceback (most recent call last):
  File "D:\AppGallery\Anaconda\envs\d2l\lib\site-packages\ipykernel\iostream.py", line 655, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 58745.6972    0  881 152693.613 58745.6972  61.5%     -    7s
     0     0 58748.1305    0  911 152693.613 58748.1305  61.5%     -    8s
     0     0 58962.3284    0  956 152693.613 58962.3284  61.4%     -   11s
     0     0 59087.5733    0  936 152693.613 59087.5733  61.3%     -   13s
     0     0 59200.9705    0  974 152693.613 59200.9705  61.2%     -   19s
     0     0 59357.0772    0  976 152693.613 59357.0772  61.1%     -   29s


KeyboardInterrupt: 

Exception ignored in: 'gurobipy._core.logcallbackstub'
Traceback (most recent call last):
  File "D:\AppGallery\Anaconda\envs\d2l\lib\site-packages\ipykernel\iostream.py", line 655, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 



Explored 1 nodes (104650 simplex iterations) in 60.11 seconds (102.48 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 152694 

Time limit reached
Best objective 1.526936131601e+05, best bound 5.935707718047e+04, gap 61.1267%
Solution Status: 9
Objective Value: 152693.61

Port Scheduling Solution:
Vessel Assignments:
  Vessel 0: Unassigned
  Vessel 1: Unassigned
  Vessel 2: Unassigned
  Vessel 3: Unassigned
  Vessel 4: Unassigned
  Vessel 5: Unassigned
  Vessel 6: Unassigned
  Vessel 7: Unassigned
  Vessel 8: Unassigned
  Vessel 9: Unassigned
  Vessel 10: Unassigned
  Vessel 11: Unassigned
  Vessel 12: Unassigned
  Vessel 13: Unassigned
  Vessel 14: Unassigned
  Vessel 15: Unassigned
  Vessel 16: Unassigned
  Vessel 17: Unassigned
  Vessel 18: Unassigned
  Vessel 19: Unassigned
  Vessel 20: Unassigned
  Vessel 21: Unassigned
  Vessel 22: Unassigned
  Vessel 23: Unassigned
  Vessel 24: Unassigned
  Vessel 25: Unassigned
  Vessel 26: Unassigned
  Vessel 27